In [1]:
import pandas as pd

In [5]:
df = pd.read_excel("./Data/Coal/Global-Coal-Mine-Tracker-April-2024.xlsx", sheet_name="Global Coal Mine Tracker (Non-C")
df.head()

,GEM Mine ID,MSHA ID,Country,Mine Name,Mine Name AKAs,Mine Name (Non-ENG),GEM Wiki URLs,Status,Status Detail,Project Type,...,Location Accuracy,"Primary Consumer, Destination","Coal Plant, Steel Plant, Terminal","Coal Plant, Steel Plant, Terminal GEM Wiki",Reported Coal Mine Methane Emissions (thousand tonnes CO2e),Year of Reported Coal Mine Methane Emissions,GEM Coal Mine Methane Emissions Estimate (MCM/yr),GEM Coal Mine Methane Emissions Estimate (M tonnes/yr),CMM Emissions (CO2e 20 years),CMM Emissions (CO2e 100 years)
0,M0001,NaN,Afghanistan,Shadashak Coal Mine,NaN,NaN,https://www.gem.wiki/Shadashak_Coal_Mine,Operating,NaN,NaN,...,Approximate,NaN,NaN,NaN,NaN,NaN,24.1,0.016,1.3,0.5
1,M0002,NaN,Australia,Airly Coal Mine,NaN,NaN,https://www.gem.wiki/Airly_Mine,Operating,NaN,NaN,...,Exact,Coal Plant,Mount Piper power station,https://www.gem.wiki/Mount_Piper_power_station,0.030,2022.0,25.8,0.017,1.4,0.5
2,M0004,NaN,Australia,Angus Place Coal Mine,Angus Place Mine Extension Project (MPMEP),NaN,https://www.gem.wiki/Angus_Place_Colliery,Cancelled,Announced,Expansion,...,Exact,NaN,NaN,NaN,NaN,NaN,21.6,0.014,1.2,0.4
3,M0004,NaN,Australia,Angus Place Coal Mine,NaN,NaN,https://www.gem.wiki/Angus_Place_Colliery,Mothballed,Care and Maintenance,NaN,...,Exact,NaN,NaN,NaN,0.539,2022.0,0,0,0,0
4,M0005,NaN,Australia,Appin Coal Mine,"West Cliff, Tower Colliery, Appin North, Appin...",NaN,https://www.gem.wiki/Appin_mine,Operating,NaN,NaN,...,Exact,Terminal (Export),Port Kembla Coal Terminal,https://www.gem.wiki/Port_Kembla_Coal_Terminal,NaN,NaN,89.2,0.06,4.9,1.8


In [ ]:
# Step 4: Drop columns
def drop_columns(df):
    columns_to_keep = ['GEM Mine ID', 'Mine Name', 'Latitude', 'Longitude', 'Country', 'Production (Mtpa)', 'Year of Production', 'Day', 'Hour']  #there's less columns we keep than drop
    columns_to_drop = set(df.columns) - set(columns_to_keep)
    return df.drop(columns=columns_to_drop)